In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import io
import rdkit
from rdkit.Chem import AllChem, Draw, rdMolDescriptors

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml(
    "/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml"
)
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(
    CONFIG_ELN["url"], CONFIG_ELN["token"]
)

In [ ]:
OPENBIS_SESSION.get_object("20250528102546698-10286").props.all()

In [ ]:
collection_options = [
    (collection.props["name"], collection.identifier)
    for collection in OPENBIS_SESSION.get_collections()
    if collection.project.space.code == "MATERIALS"
]
collection_options.insert(0, ("Select a collection...", -1))
edit_tab_collection_selector = utils.Dropdown(
    options=collection_options,
    value=-1,
    style={"description_width": "initial"},
)

register_tab_collection_selector = utils.Dropdown(
    options=collection_options,
    value=-1,
    style={"description_width": "initial"},
)

object_list_options = [("Select an object...", -1)]
object_list_selector = utils.Dropdown(
    options=object_list_options,
    value=-1,
    style={"description_width": "initial"},
)

In [ ]:
inventory_types = [
    ("Select inventory object type...", -1),
    ("Crystal", "Crystal"),
    ("Crystal Concept", "CrystalConcept"),
    ("Molecule", "Molecule"),
    ("Molecule Concept", "MoleculeConcept"),
    ("Reaction Product Concept", "ReactionProductConcept"),
    ("Reaction Product", "ReactionProduct"),
    ("2D Layer Material", "TwoDLayerMaterial"),
]

inventory_type_dropdown = utils.Dropdown(options=inventory_types, value=-1)

In [ ]:
support_files_widget = ipw.FileUpload(multiple=True)
register_object_widget = ipw.VBox()
register_object_tab_widgets = ipw.VBox(
    [
        ipw.HTML(value="<h3><b>Select collection</b></h3>"),
        register_tab_collection_selector,
        ipw.HTML(value="<h3><b>Select object type</b></h3>"),
        inventory_type_dropdown,
        ipw.HTML(value="<h3><b>Properties</b></h3>"),
        register_object_widget,
        ipw.HTML(value="<h3><b>Support files</b></h3>"),
        support_files_widget,
    ]
)

edit_object_widget = ipw.VBox()
edit_object_tab_widgets = ipw.VBox(
    [
        ipw.HTML(value="<h3><b>Select collection</b></h3>"),
        edit_tab_collection_selector,
        ipw.HTML(value="<h3><b>Select object</b></h3>"),
        object_list_selector,
        ipw.HTML(value="<h3><b>Properties</b></h3>"),
        edit_object_widget,
        ipw.HTML(value="<h3><b>Support files</b></h3>"),
        support_files_widget,
    ]
)

cdxml_uploader_header = ipw.HTML(value="<h3><b>Upload CDXML file</b></h3>")
cdxml_uploader_widget = ipw.FileUpload(multiple=False, accept=".cdxml")
cdxml_molecule_structure = ipw.Image(
    format="jpg",
    width="420px",
    height="450px",
    layout=ipw.Layout(border="solid 1px #cccccc"),
)
cdxml_structure_header = ipw.HTML(value="<h3><b>Molecule structure</b></h3>")

tabs_names = ["Register objects", "Search/edit objects"]
interface_tabs = ipw.Tab()
interface_tabs.children = [register_object_tab_widgets, edit_object_tab_widgets]
for i, e in enumerate(tabs_names):
    interface_tabs.set_title(i, tabs_names[i])

In [ ]:
increase_buttons_size = utils.HTML(data="".join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Save",
    icon="save",
    layout=ipw.Layout(width="100px", height="50px"),
)
quit_button = utils.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data='window.location.replace("home.ipynb")'))


def select_inventory_type(change):
    register_object_tab_widgets.children = [
        ipw.HTML(value="<h3><b>Select collection</b></h3>"),
        register_tab_collection_selector,
        ipw.HTML(value="<h3><b>Select object type</b></h3>"),
        inventory_type_dropdown,
        ipw.HTML(value="<h3><b>Properties</b></h3>"),
        register_object_widget,
        ipw.HTML(value="<h3><b>Support files</b></h3>"),
        support_files_widget,
    ]

    # Reset support files uploader
    support_files_widget.value.clear()
    support_files_widget._counter = 0

    inventory_type = inventory_type_dropdown.value
    if inventory_type == -1:
        register_object_widget.children = []
        return

    object_props_widgets = widgets.OpenbisObjectWidget(inventory_type, DATA_MODEL)

    if inventory_type in ["Crystal", "Molecule", "TwoDLayerMaterial"]:
        checkbox_on_change(
            object_props_widgets.properties_widgets["hazardous"]["value_widget"],
            object_props_widgets.properties_widgets["hazardous_specification"][
                "value_widget"
            ],
        )
        checkbox_on_change(
            object_props_widgets.properties_widgets["other_storage_condition"][
                "value_widget"
            ],
            object_props_widgets.properties_widgets[
                "other_storage_condition_specification"
            ]["value_widget"],
        )

    if inventory_type == "MoleculeConcept":
        # Reset CDXML file uploader
        cdxml_uploader_widget.value.clear()
        cdxml_uploader_widget._counter = 0  # This is not working???

        # Reset Image widget
        cdxml_molecule_structure.value = b""

        register_object_tab_widgets_children = list(
            register_object_tab_widgets.children
        )
        register_object_tab_widgets.children = (
            register_object_tab_widgets_children[:4]
            + [
                cdxml_uploader_header,
                cdxml_uploader_widget,
                cdxml_structure_header,
                cdxml_molecule_structure,
            ]
            + register_object_tab_widgets_children[4:]
        )

        def load_molecule_structure(change):
            for filename in cdxml_uploader_widget.value:
                file_info = cdxml_uploader_widget.value[filename]
                utils.save_file(file_info["content"], "structures/structure.cdxml")

            cdxml_molecule = utils.read_file("structures/structure.cdxml")
            molecules = rdkit.Chem.MolsFromCDXML(cdxml_molecule)

            if len(molecules) == 1:
                mol = molecules[0]  # Get first molecule
                mol_chemical_formula = rdMolDescriptors.CalcMolFormula(
                    mol
                )  # Sum Formula
                mol_smiles = rdkit.Chem.MolToSmiles(mol)  # Canonical Smiles
                chem_mol = rdkit.Chem.MolFromSmiles(mol_smiles)

                if chem_mol is not None:
                    AllChem.Compute2DCoords(
                        chem_mol
                    )  # Add coords to the atoms in the molecule
                    img = Draw.MolToImage(chem_mol)
                    buffer = io.BytesIO()
                    img.save(buffer, format="PNG")
                    cdxml_molecule_structure.value = buffer.getvalue()
                    img.save("structures/structure.png")
                else:
                    print("Cannot generate molecule image.")

                object_props_widgets.properties_widgets["sum_formula"][
                    "value_widget"
                ].value = mol_chemical_formula
                object_props_widgets.properties_widgets["smiles"][
                    "value_widget"
                ].value = mol_smiles

            elif len(molecules) > 1:
                print(f"There are more than one molecule in the file: {filename}")
            else:
                print(f"There are no molecules in the file: {filename}")

        cdxml_uploader_widget.observe(load_molecule_structure, names="value")

    register_object_widget.children = [object_props_widgets]


def create_object_openbis(b):
    inventory_type = inventory_type_dropdown.value
    if inventory_type == -1:
        return

    if inventory_type == "Molecule":
        object_selector = (
            register_object_widget.children[0]
            .properties_widgets["molecule_concept"]["value_widget"]
            .dropdown
        )

        if object_selector.value == -1:
            display(utils.Javascript(data="alert('Select a molecule concept.')"))
            return

        # Check if the molecule is already in openBIS
        molecules_empa_id_openbis = [
            f"{molecule.props['empa_number']}-{molecule.props['batch']}"
            for molecule in utils.get_openbis_objects(OPENBIS_SESSION, type="MOLECULE")
        ]
        object_properties = register_object_widget.children[0].get_values()

        if "empa_number" not in object_properties:
            display(utils.Javascript(data="alert('Empa number was not specified.')"))
            return
        else:
            object_properties["empa_number"] = int(object_properties["empa_number"])

        if "batch" not in object_properties:
            display(utils.Javascript(data="alert('Batch was not specified.')"))
            return

        molecule_empa_id = (
            f"{object_properties['empa_number']}-{object_properties['batch']}"
        )
        if molecule_empa_id in molecules_empa_id_openbis:
            display(utils.Javascript(data="alert('Molecule is already in openBIS!')"))
            return

        openbis_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type="MOLECULE",
            collection="/MATERIALS/MOLECULES/PRECURSOR_COLLECTION",
            props=object_properties,
        )

    elif inventory_type == "MoleculeConcept":
        object_properties = register_object_widget.children[0].get_values()

        openbis_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type="MOLECULE_CONCEPT",
            collection="/MATERIALS/MOLECULES/PRECURSOR_COLLECTION",
            props=object_properties,
        )

        utils.upload_datasets(
            OPENBIS_SESSION, openbis_object, cdxml_uploader_widget, "RAW_DATA"
        )

    utils.upload_datasets(
        OPENBIS_SESSION, openbis_object, support_files_widget, "RAW_DATA"
    )
    display(utils.Javascript(data="alert('Upload successful!')"))


def checkbox_on_change(checkbox, textbox):
    textbox.disabled = True

    def enable_textbox(change):
        if checkbox.value:
            textbox.disabled = False
        else:
            textbox.disabled = True

    checkbox.observe(enable_textbox, names="value")


def load_object_list(change):
    collection_id = edit_tab_collection_selector.value
    if collection_id == -1:
        object_list_selector.options = object_list_options
        object_list_selector.value = -1
        return

    else:
        object_list_options = [("Select an object...", -1)]
        object_list_options += [
            (openbis_object.props["name"], openbis_object.permId)
            for openbis_object in OPENBIS_SESSION.get_objects(collection=collection_id)
        ]
        object_list_selector.options = object_list_options


def load_object(change):
    object_id = object_list_selector.value
    if object_id == -1:
        return

    else:
        openbis_object = OPENBIS_SESSION.get_object(object_id)
        for key, item in DATA_MODEL["classes"].items():
            class_annotations = item.get("annotations", None)
            if class_annotations is None:
                continue
            else:
                class_openbis_code = class_annotations.get("openbis_code", None)
                if class_openbis_code is None:
                    continue
                else:
                    if class_openbis_code == openbis_object.code[:4]:
                        inventory_type = key
                        print(openbis_object.props.all())
                        object_props_widgets = widgets.OpenbisObjectWidget(
                            key, DATA_MODEL
                        )

                        if inventory_type in [
                            "Crystal",
                            "Molecule",
                            "TwoDLayerMaterial",
                        ]:
                            checkbox_on_change(
                                object_props_widgets.properties_widgets["hazardous"][
                                    "value_widget"
                                ],
                                object_props_widgets.properties_widgets[
                                    "hazardous_specification"
                                ]["value_widget"],
                            )
                            checkbox_on_change(
                                object_props_widgets.properties_widgets[
                                    "other_storage_condition"
                                ]["value_widget"],
                                object_props_widgets.properties_widgets[
                                    "other_storage_condition_specification"
                                ]["value_widget"],
                            )

                        if inventory_type == "MoleculeConcept":
                            # Reset CDXML file uploader
                            cdxml_uploader_widget.value.clear()
                            cdxml_uploader_widget._counter = 0  # This is not working???

                            # Reset Image widget
                            cdxml_molecule_structure.value = b""

                            edit_object_tab_widgets_children = list(
                                edit_object_tab_widgets.children
                            )
                            edit_object_tab_widgets.children = (
                                edit_object_tab_widgets_children[:4]
                                + [
                                    cdxml_uploader_header,
                                    cdxml_uploader_widget,
                                    cdxml_structure_header,
                                    cdxml_molecule_structure,
                                ]
                                + edit_object_tab_widgets_children[4:]
                            )

                            def load_molecule_structure(change):
                                for filename in cdxml_uploader_widget.value:
                                    file_info = cdxml_uploader_widget.value[filename]
                                    utils.save_file(
                                        file_info["content"],
                                        "structures/structure.cdxml",
                                    )

                                cdxml_molecule = utils.read_file(
                                    "structures/structure.cdxml"
                                )
                                molecules = rdkit.Chem.MolsFromCDXML(cdxml_molecule)

                                if len(molecules) == 1:
                                    mol = molecules[0]  # Get first molecule
                                    mol_chemical_formula = (
                                        rdMolDescriptors.CalcMolFormula(mol)
                                    )  # Sum Formula
                                    mol_smiles = rdkit.Chem.MolToSmiles(
                                        mol
                                    )  # Canonical Smiles
                                    chem_mol = rdkit.Chem.MolFromSmiles(mol_smiles)

                                    if chem_mol is not None:
                                        AllChem.Compute2DCoords(
                                            chem_mol
                                        )  # Add coords to the atoms in the molecule
                                        img = Draw.MolToImage(chem_mol)
                                        buffer = io.BytesIO()
                                        img.save(buffer, format="PNG")
                                        cdxml_molecule_structure.value = (
                                            buffer.getvalue()
                                        )
                                        img.save("structures/structure.png")
                                    else:
                                        print("Cannot generate molecule image.")

                                    object_props_widgets.properties_widgets[
                                        "sum_formula"
                                    ]["value_widget"].value = mol_chemical_formula
                                    object_props_widgets.properties_widgets["smiles"][
                                        "value_widget"
                                    ].value = mol_smiles

                                elif len(molecules) > 1:
                                    print(
                                        f"There are more than one molecule in the file: {filename}"
                                    )
                                else:
                                    print(
                                        f"There are no molecules in the file: {filename}"
                                    )

                            cdxml_uploader_widget.observe(
                                load_molecule_structure, names="value"
                            )

                        edit_object_widget.children = [object_props_widgets]

In [ ]:
inventory_type_dropdown.observe(select_inventory_type, names="value")
edit_tab_collection_selector.observe(load_object_list, names="value")
object_list_selector.observe(load_object, names="value")

# Register/edit inventory objects

In [ ]:
display(interface_tabs)

## Save object

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_object_openbis)
quit_button.on_click(close_notebook)